In [32]:
#vdemonly 
# read thru and combine with run summary
#build a pipeline to assemble all the images
#Amazon S3/planet-snowcover-experiments/planet-snowcover-2020-03-06-21-21-30-038
import pprint
import s3fs
import boto3
import io
from re import match
%matplotlib inline
import rasterio as rio

p = pprint.PrettyPrinter()

fs = s3fs.S3FileSystem(session = boto3.Session(profile_name = 'esip'))
#dem only
imagery_searchpath = 's3://planet-snowcover-archive'
#print("Searching for imagery...({})".format(imagery_searchpath))
imagery_candidates = fs.ls(imagery_searchpath)
print("candidates:")
#p.pprint(imagery_candidates)
# filter only the test scenes tiles
archived_files = [link for link  in imagery_candidates]
#p.pprint(archived_files)

candidates:


In [33]:
archived_files[0]

'planet-snowcover-archive/PSScene4Band_20170125_175925_0e20_all_udm2.zip'

In [34]:
list_copied_files=[]
with open('list_of_uploadedfiles.txt') as f:
     for line in f:
        file_copied = line.split(" ")[1]
        file_copied = file_copied.split('\n')[0]
        list_copied_files.append(file_copied)
        

In [54]:
len(list_copied_files)

22334

In [36]:
archived_files[0:10]

['planet-snowcover-archive/PSScene4Band_20170125_175925_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175926_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175927_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175928_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175929_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175930_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175931_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175932_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175933_0e20_all_udm2.zip',
 'planet-snowcover-archive/PSScene4Band_20170125_175934_0e20_all_udm2.zip']

In [37]:
#aws s3api restore-object --bucket planet-snowcover-archive --key  PSScene4Band_20190503_181900_100a_all_udm2.zip --restore-request '{"Days":25,"GlacierJobParameters":{"Tier":"Standard"}}'
archivefiles =[]
for link in archived_files:
    if link.endswith('.zip'):
        #print(link.split('/')[1])
        folder = link.split('/')[0]
        archive_file= link.split('/')[1]
        archivefiles.append(archive_file)
        

In [38]:
len(archivefiles)

22534

In [39]:
# files not copied
files_not_copied = []
for file in archivefiles:
    if file not in list_copied_files:
        files_not_copied.append(file)

In [55]:
len(files_not_copied)

200

In [51]:
files_not_copied[0:10]

['PSScene4Band_20190119_181807_1025_all_udm2.zip',
 'PSScene4Band_20190119_181810_1025_all_udm2.zip',
 'PSScene4Band_20190119_181812_1025_all_udm2.zip',
 'PSScene4Band_20190120_164854_0f36_all_udm2.zip',
 'PSScene4Band_20190120_164855_0f36_all_udm2.zip',
 'PSScene4Band_20190120_164856_0f36_all_udm2.zip',
 'PSScene4Band_20190120_164857_0f36_all_udm2.zip',
 'PSScene4Band_20190120_164858_0f36_all_udm2.zip',
 'PSScene4Band_20190120_164859_0f36_all_udm2.zip',
 'PSScene4Band_20190120_164900_0f36_all_udm2.zip']

In [56]:
import boto3
 
s3 = boto3.resource('s3')
bucket = s3.Bucket('planet-snowcover-archive')
restore_requestDict = {
'Days': 23 # Number of days object is kept in S3 before it is removed
}
countRestored=0
countInRestore=0
countNotInRestore=0
bucket_name = 'planet-snowcover-archive'
for object in bucket.objects.all():
    filename = object.key
    if filename in files_not_copied:
        object = s3.Object(object.bucket_name, object.key)
        #if it's in Glacier
        if object.storage_class == "GLACIER":
            #and not currently being restored:
            if object.restore is None:
                print("Not in restore queue --> ",object.key)
                countNotInRestore= countNotInRestore+1
                object.restore_object(RestoreRequest={
                    'Days': 12,
                    'GlacierJobParameters': {
                        'Tier': 'Bulk'
                    }
                })
                print("Added to restore queue --> ",object.key)
            #list ones currently queued for restore
            elif 'ongoing-request="true"' in object.restore:
                print("Already in restore queue --> "+ object.key)
                countInRestore= countInRestore+1
            #list ones already restored
            elif 'ongoing-request="false"' in object.restore:
                print("Already restored --> "+object.key)
                countRestored= countRestored+1
print('no of restored', countRestored)
print('no of in restoreq',countInRestore)
print('no not in restoreq',countNotInRestore)

Already restored --> PSScene4Band_20190119_181807_1025_all_udm2.zip
Already restored --> PSScene4Band_20190119_181810_1025_all_udm2.zip
Already restored --> PSScene4Band_20190119_181812_1025_all_udm2.zip
Already restored --> PSScene4Band_20190120_164854_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_164855_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_164856_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_164857_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_164858_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_164859_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_164900_0f36_all_udm2.zip
Already restored --> PSScene4Band_20190120_165601_0f24_all_udm2.zip
Already restored --> PSScene4Band_20190120_165602_0f24_all_udm2.zip
Already restored --> PSScene4Band_20190120_172518_0f12_all_udm2.zip
Already restored --> PSScene4Band_20190120_172519_0f12_all_udm2.zip
Already restored --> PSScene4Band_20190126_18190

In [57]:
import boto3
session = boto3.Session(profile_name='esip')
# Any clients created from this session will use credentials
# from the [dev] section of ~/.aws/credentials.
dev_s3_client = session.client('s3')
countCopied = 0
for link in files_not_copied:
    folder = 'planet-snowcover-archive'
    archive_file= link
    with open(archive_file, 'wb') as f:
        dev_s3_client.download_fileobj(folder, archive_file, f)
    !rclone copy {archive_file} gdrive-web:Planet_Archive
    !rm {archive_file}
    countCopied = countCopied+1
    print("copied",archive_file )

copied PSScene4Band_20190119_181807_1025_all_udm2.zip
copied PSScene4Band_20190119_181810_1025_all_udm2.zip
copied PSScene4Band_20190119_181812_1025_all_udm2.zip
copied PSScene4Band_20190120_164854_0f36_all_udm2.zip
copied PSScene4Band_20190120_164855_0f36_all_udm2.zip
copied PSScene4Band_20190120_164856_0f36_all_udm2.zip
copied PSScene4Band_20190120_164857_0f36_all_udm2.zip
copied PSScene4Band_20190120_164858_0f36_all_udm2.zip
copied PSScene4Band_20190120_164859_0f36_all_udm2.zip
copied PSScene4Band_20190120_164900_0f36_all_udm2.zip
copied PSScene4Band_20190120_165601_0f24_all_udm2.zip
copied PSScene4Band_20190120_165602_0f24_all_udm2.zip
copied PSScene4Band_20190120_172518_0f12_all_udm2.zip
copied PSScene4Band_20190120_172519_0f12_all_udm2.zip
copied PSScene4Band_20190126_181905_1014_all_udm2.zip
copied PSScene4Band_20190126_181906_1014_all_udm2.zip
copied PSScene4Band_20190126_181943_1035_all_udm2.zip
copied PSScene4Band_20190126_181944_1035_all_udm2.zip
copied PSScene4Band_20190126

In [3]:
import boto3
 
s3 = boto3.resource('s3')
bucket = s3.Bucket('planet-snowcover-archive')
restore_requestDict = {
'Days': 23, # Number of days object is kept in S3 before it is removed
'GlacierJobParameters': {
'Tier': 'Bulk' # can also be 'Expedited' or 'Standard' see Glacier restore options for explanation
},
}
countRestored=0
countInRestore=0
countNotInRestore=0
for object in bucket.objects.all():
    #folder, filename = object.key.split("/")
    object = s3.Object(object.bucket_name, object.key)
    #if it's in Glacier
    if object.storage_class == "GLACIER":
        #and not currently being restored:
        if object.restore is None:
            print("Not in restore queue --> ",resp)
            countNotInRestore= countNotInRestore+1
            #object.restore_object(Bucket=object.bucket_name, Key=object.key,restoreRequest=restore_requestDict) #this is where I crash and burn!!!
        #list ones currently queued for restore
        elif 'ongoing-request="true"' in object.restore:
            print("Already in restore queue --> "+ object.key)
            countInRestore= countInRestore+1
        #list ones already restored
        elif 'ongoing-request="false"' in object.restore:
            print("Already restored --> "+object.key)
            countRestored= countRestored+1
print('no of restored', countRestored)
print('no of in restoreq',countInRestore)
print('no not in restoreq',countNotInRestore)

Already restored --> PSScene4Band_20170125_175925_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175926_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175927_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175928_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175929_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175930_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175931_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175932_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175933_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175934_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175935_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170125_175936_0e20_all_udm2.zip
Already restored --> PSScene4Band_20170329_170838_0f18_all_udm2.zip
Already restored --> PSScene4Band_20170408_165902_1009_all_udm2.zip
Already restored --> PSScene4Band_20170408_16590

In [1]:
import boto3
 
s3 = boto3.resource('s3')
bucket = s3.Bucket('planet-snowcover-archive')
restore_requestDict = {
'Days': 23, # Number of days object is kept in S3 before it is removed
'GlacierJobParameters': {
'Tier': 'Bulk' # can also be 'Expedited' or 'Standard' see Glacier restore options for explanation
},
}
for object in bucket.objects.all():
    #folder, filename = object.key.split("/")
    object = s3.Object(object.bucket_name, object.key)
    #if it's in Glacier
    if object.storage_class == "GLACIER":
        #and not currently being restored:
        if object.restore is None:
            print("Requesting restore for --> "+object.key) 
            resp = bucket.meta.client.restore_object(
                    Bucket=object.bucket_name,
                    Key=object.key,
                    RestoreRequest={'Days': 23}
            )
            print("Put in restore queue --> ",resp)
            #object.restore_object(Bucket=object.bucket_name, Key=object.key,restoreRequest=restore_requestDict) #this is where I crash and burn!!!
        #list ones currently queued for restore
        elif 'ongoing-request="true"' in object.restore:
            print("Already in restore queue --> "+ object.key)
        #list ones already restored
        elif 'ongoing-request="false"' in object.restore:
            print("Already restored --> "+object.key)

Already restored --> PSScene4Band_20170125_175925_0e20_all_udm2.zip
Requesting restore for --> PSScene4Band_20170125_175926_0e20_all_udm2.zip
Put in restore queue -->  {'ResponseMetadata': {'RequestId': '85E20D8C9949587C', 'HostId': 'E5+04z2vEbWU8y2t1awse21mpBUCyiMrh+B4bcZAJb0k7/ywjQmvz6i0tE7QR6F4N6XN1xiVMlk=', 'HTTPStatusCode': 202, 'HTTPHeaders': {'x-amz-id-2': 'E5+04z2vEbWU8y2t1awse21mpBUCyiMrh+B4bcZAJb0k7/ywjQmvz6i0tE7QR6F4N6XN1xiVMlk=', 'x-amz-request-id': '85E20D8C9949587C', 'date': 'Sat, 09 May 2020 08:22:01 GMT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
Requesting restore for --> PSScene4Band_20170125_175927_0e20_all_udm2.zip
Put in restore queue -->  {'ResponseMetadata': {'RequestId': 'E753F5760370D825', 'HostId': 'w8d519a4SiyxbAUmK1DM0F5cVg3mt0Xg+BVVLTYcEvJg512STAXGOD37e1wDUYpamYCE7mMjMV4=', 'HTTPStatusCode': 202, 'HTTPHeaders': {'x-amz-id-2': 'w8d519a4SiyxbAUmK1DM0F5cVg3mt0Xg+BVVLTYcEvJg512STAXGOD37e1wDUYpamYCE7mMjMV4=', 'x-amz-request-id': 'E753F57

In [ ]:
rclone ls  gdrive-web:Planet_Archive | wc -l

In [3]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

In [4]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
# These define the bucket and object to read
bucketname = 'planet-snowcover-experiments'
file_to_read = 'planet-snowcover-2020-03-12-02-43-15-610/test_ids.txt' 

#Create a file object using the bucket and object key. 
fileobj = s3_client.get_object(
    Bucket=bucketname,
    Key=file_to_read
    ) 
# open the file object and read it into the variable filedata. 
filedata = fileobj['Body'].read()

# file data will be a binary stream.  We have to decode it 
contents = filedata.decode('utf-8') 

# Once decoded, you can treat the file as plain text if appropriate 
#print(contents)

In [5]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
# These define the bucket and object to read
bucketname = 'planet-snowcover-experiments'
file_to_read = 'planet-snowcover-2020-03-06-21-21-30-038/test_ids.txt' 

#Create a file object using the bucket and object key. 
fileobj = s3_client.get_object(
    Bucket=bucketname,
    Key=file_to_read
    ) 
# open the file object and read it into the variable filedata. 
filedata = fileobj['Body'].read()

# file data will be a binary stream.  We have to decode it 
contents = filedata.decode('utf-8') 

# Once decoded, you can treat the file as plain text if appropriate 
#print(contents)

In [6]:
for link in imagery_candidates:
    print(link)
    #sub_candidates = fs.ls(link)
    #print(sub_candidates)
    #dev_s3_client.list_objects(link) 
    s3 = boto3.resource('s3')
    s3_client = boto3.client('s3')

    bucket=link.partition('/')[0] 
    my_bucket = s3.Bucket(bucket)
    for my_bucket_object in my_bucket.objects.filter(Prefix=link.partition('/')[2]):
        #print('s3://{0}/{1}'.format(my_bucket.name, my_bucket_object.key))

SyntaxError: unexpected EOF while parsing (<ipython-input-6-fb6b3f60fae9>, line 12)